In [54]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()
print(OPENAI_API_KEY)



sk-proj-sxp1CfbM20xTzYUlEi2tT3BlbkFJhq11qNZDGC4AakfQOYkB


In [55]:
import nest_asyncio

nest_asyncio.apply()

In [56]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [57]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [58]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [59]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [60]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [61]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to MetaGPT"),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from the MetaGPT paper."),
)

In [62]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True,
)

In [63]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document discusses MetaGPT, a meta-programming framework that utilizes SOPs to enhance multi-agent systems based on LLMs. It outlines the development process involving various agents like Product Managers, Architects, Engineers, Project Managers, and QA Engineers. MetaGPT demonstrates superior performance in code generation tasks compared to other frameworks. The document provides insights into the software development lifecycle of a "Drawing App" using MetaGPT, highlighting the roles of different agents and the use of Python libraries for GUI creation. It also addresses limitations, ethical concerns, and challenges associated with MetaGPT while showcasing its capabilities in automating software development tasks efficiently.


In [64]:
print(len(response.source_nodes))

34


In [48]:
response = query_engine.query("How do agents share information with other agents?")
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context, which is necessary for understanding how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and also subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information directly without the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


In [96]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [52]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, so choice 2 is most relevant for retrieving this information..
The ablation study results provide insights into the impact of removing certain components or features from a system or model. This analysis helps in understanding the contribution and significance of individual elements towards the overall performance or functionality of the system.


# Lesson 2

In [66]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [67]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


In [68]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [69]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [70]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-18
last_modified_date: 2024-05-17

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [71]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [72]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts([{"key": "page_label", "value": "2"}]),
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?",
)

In [73]:
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with 85.9% and 87.7% in Pass@1, outperforming other popular frameworks like AutoGPT, LangChain, AgentVerse, and ChatDev. Additionally, MetaGPT demonstrates robustness and efficiency by achieving a 100% task completion rate in experimental evaluations, highlighting its effectiveness in handling higher levels of software complexity and offering extensive functionality.


In [74]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-17'}


In [75]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(query: str, page_numbers: List[str]) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(name="vector_tool", fn=vector_query)

In [76]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool],
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with 85.9% and 87.7% in Pass@1. It stands out in handling higher levels of software complexity and offering extensive functionality, demonstrating a 100% task completion rate in experimental evaluations.


In [77]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-17'}


In [78]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("Useful if you want to get a summary of MetaGPT"),
)

In [79]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various aspects. For example, MetaGPT achieves a higher score in executability, takes less time for execution, requires more tokens but uses fewer tokens to generate one line of code compared to ChatDev. Additionally, MetaGPT demonstrates better performance in code statistics and human revision cost when compared to ChatDev.


In [80]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-17'}


In [81]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], "What is a summary of the paper?", verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "The paper discusses the impact of climate change on biodiversity and ecosystems."}
=== Function Output ===
The paper does not discuss the impact of climate change on biodiversity and ecosystems.


# Lesson 3

In [113]:
from typing import Optional
# this should be a utils function (in utils.py)


def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(query: str, page_numbers: Optional[List[str]] = None) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts, condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}", fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

In [115]:
vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [116]:
import sys
print(sys.path)

['/Users/alexanderphan_1/Developer/ilya-papers/backend', '/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/alexanderphan_1/Developer/ilya-papers/backend/venv/lib/python3.12/site-packages']


In [117]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [118]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [119]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for creating the Product Requirement Document and conducting competitive analysis. The Architect is in charge of designing technical specifications and system architecture. The Project Manager breaks down the project into tasks for the team to execute. Engineers implement the code based on the provided specifications. Lastly, the QA Engineer is responsible for generating unit tests and ensuring the software's quality. Each role contributes significantly to the collaborative software development process within MetaGPT.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {

In [120]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-18
last_modified_date: 2024-05-17

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [121]:
response = agent.chat("Tell me about the evaluation datasets used.")

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. The HumanEval dataset consists of 164 handwritten programming tasks, while the MBPP dataset consists of 427 Python tasks. The SoftwareDev dataset contains 70 representative software development tasks covering various scopes such as mini-games, image processing algorithms, and data visualization.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset. HumanEval comprises 164 handwritten programming tasks, MBPP consists of 427 Python tasks, and the SoftwareDev dataset includes 70 representative software development tasks covering mini-games, image processing algorithms, and data visualization.


In [122]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over the HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% results over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved impressive results of 85.9% and 87.7% over the HumanEval dataset.


In [123]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [124]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [125]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for creating the Product Requirement Document (PRD). The Architect designs technical specifications and system architecture diagrams. The Project Manager breaks down the project into tasks and assigns them to Engineers. Engineers implement development tasks based on the assigned tasks. The QA Engineer generates unit test code, reviews the code output, and ensures high-quality software by identifying and fixing bugs.


In [126]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task df446033-db22-4214-96da-f3e599959822: 1
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for creating the Product Requirement Document (PRD). The Architect designs technical specifications and system architecture diagrams. The Project Manager breaks down the project into tasks and assigns them to Engineers. Engineers implement development tasks based on the assigned tasks. The QA Engineer generates unit test code, reviews the code output, and ensures high-quality software by identifying and fixing bugs.


In [127]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task df446033-db22-4214-96da-f3e599959822: 1


TaskStep(task_id='df446033-db22-4214-96da-f3e599959822', step_id='2fb681d5-c6b5-467d-ba0f-4a8e898c577e', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [128]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a shared message pool and a subscription mechanism. The shared message pool allows agents to exchange structured messages directly, facilitating transparent communication. Agents can subscribe to relevant messages based on their profiles, ensuring efficient communication and information exchange within the system. This structured communication interface helps prevent information overload and enhances communication efficiency among the agents.


In [129]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, agents share information through a shared message pool and a subscription mechanism. The shared message pool enables agents to exchange structured messages directly, facilitating transparent communication. Agents can subscribe to relevant messages based on their profiles, ensuring efficient communication and information exchange within the system. This structured communication interface helps prevent information overload and enhances communication efficiency among the agents.
True


In [130]:
response = agent.finalize_response(task.task_id)

In [131]:
print(str(response))

assistant: In MetaGPT, agents share information through a shared message pool and a subscription mechanism. The shared message pool enables agents to exchange structured messages directly, facilitating transparent communication. Agents can subscribe to relevant messages based on their profiles, ensuring efficient communication and information exchange within the system. This structured communication interface helps prevent information overload and enhances communication efficiency among the agents.


# Lesson 4

In [213]:
urls = [
    "https://nlp.seas.harvard.edu/annotated-transformer/",
    "https://scottaaronson.blog/?p=762",
    "https://karpathy.github.io/2015/05/21/rnn-effectiveness/",
    "https://colah.github.io/posts/2015-08-Understanding-LSTMs/",
    "https://cs231n.github.io/",
    "https://arxiv.org/pdf/1409.2329.pdf",
    "https://www.cs.toronto.edu/~hinton/absps/colt93.pdf",
    "https://arxiv.org/pdf/1506.03134.pdf",
    "https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf",
    "https://arxiv.org/pdf/1511.06391.pdf",
    "https://arxiv.org/pdf/1811.06965.pdf",
    "https://arxiv.org/pdf/1512.03385.pdf",
    "https://arxiv.org/pdf/1511.07122.pdf",
    "https://arxiv.org/pdf/1704.01212.pdf",
    "https://arxiv.org/pdf/1706.03762.pdf",
    "https://arxiv.org/pdf/1409.0473.pdf",
    "https://arxiv.org/pdf/1603.05027.pdf",
    "https://arxiv.org/pdf/1706.01427.pdf",
    "https://arxiv.org/pdf/1611.02731.pdf",
    "https://arxiv.org/pdf/1806.01822.pdf",
    "https://arxiv.org/pdf/1405.6903.pdf",
    "https://arxiv.org/pdf/1410.5401.pdf",
    "https://arxiv.org/pdf/1512.02595.pdf",
    "https://arxiv.org/pdf/2001.08361.pdf",
    "https://arxiv.org/pdf/math/0406077.pdf",
    "https://www.vetta.org/documents/Machine_Super_Intelligence.pdf",
    "https://www.lirmm.fr/~ashen/kolmbook-eng-scan.pdf",
]

# in a subdirectory called papers
papers = [
    "papers/annotated_transformer.pdf",
    "papers/first_law_of_thermodynamics.html",
    "papers/karpathy_rnn_effectiveness.html",
    "papers/colah_understanding_lstms.html",
    "papers/cs231n_2015_rnn_regularization.pdf",
    "papers/keeping_nn_simple_hinton.pdf",
    "papers/pointer_networks.pdf",
    "papers/imagenet_classification.pdf",
    "papers/order_matters_s_to_s.pdf",
    "papers/GPipe.pdf",
    "papers/deep_residual_learning_image_recognition.pdf",
    "papers/multi_scale_context_aggregation.pdf",
    "papers/nerual_quantum_chemistry.pdf",
    "papers/attention_is_all_you_need.pdf",
    "papers/neural_machine_translation.pdf",
    "papers/identity_mappings_deep_residual_networks.pdf",
    "papers/simple_nn_module.pdf",
    "papers/variational_lossy_autoencoder.pdf",
    "papers/relational_rnns.pdf",
    "papers/quantifying_rise_fall_complexity.pdf",
    "papers/neural_turing_machines.pdf",
    "papers/deep_speech_2.pdf",
    "papers/scaling_laws_for_neural_llms.pdf",
    "papers/intro_minimum_description_length_principle.pdf",
    "papers/machine_super_intelligence.pdf",
    "papers/kolmogorov_complexity.pdf",
    "papers/cnns_for_visual_recognition.html",
]



In [214]:
# from utils import get_doc_tools
# should be a utils function (utils.py)


def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(query: str, page_numbers: Optional[List[str]] = None) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts, condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}", fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool


In [215]:
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

invalid pdf header: b'<!DOC'


Getting tools for paper: papers/annotated_transformer.pdf


EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Failed to load file papers/annotated_transformer.pdf with error: RetryError[<Future at 0x29543c3b0 state=finished raised PdfStreamError>]. Skipping...
Getting tools for paper: papers/first_law_of_thermodynamics.html
Getting tools for paper: papers/karpathy_rnn_effectiveness.html
Getting tools for paper: papers/colah_understanding_lstms.html


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Getting tools for paper: papers/cs231n_2015_rnn_regularization.pdf
Failed to load file papers/cs231n_2015_rnn_regularization.pdf with error: RetryError[<Future at 0x29e4f1550 state=finished raised PdfStreamError>]. Skipping...
Getting tools for paper: papers/keeping_nn_simple_hinton.pdf
Getting tools for paper: papers/pointer_networks.pdf
Getting tools for paper: papers/imagenet_classification.pdf
Getting tools for paper: papers/order_matters_s_to_s.pdf
Getting tools for paper: papers/GPipe.pdf
Getting tools for paper: papers/deep_residual_learning_image_recognition.pdf
Getting tools for paper: papers/multi_scale_context_aggregation.pdf
Getting tools for paper: papers/nerual_quantum_chemistry.pdf
Getting tools for paper: papers/attention_is_all_you_need.pdf
Getting tools for paper: papers/neural_machine_translation.pdf
Getting tools for paper: papers/identity_mappings_deep_residual_networks.pdf
Getting tools for paper: papers/simple_nn_module.pdf
Getting tools for paper: papers/variati

In [216]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [217]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [218]:
len(initial_tools)

54

In [219]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [220]:
response = agent.query(
    "Tell me about the evaluation dataset used in rnns "
    "and then tell me about the evaluation results"
)

Added user message to memory: Tell me about the evaluation dataset used in rnns and then tell me about the evaluation results
=== Calling Function ===
Calling function: vector_tool_relational_rnns with args: {"query": "evaluation dataset"}
=== Function Output ===
Semi-supervised variational autoencoders for sequence classification.
=== Calling Function ===
Calling function: vector_tool_relational_rnns with args: {"query": "evaluation results"}
=== Function Output ===
VLAE models with AF prior and PixelCNN conditional achieved new state-of-the-art results on statically binarized MNIST. Additionally, on dynamically binarized MNIST, VLAE outperformed other models in terms of test NLL. On the OMNIGLOT dataset, VLAE showed improved performance compared to other models, especially when fine-tuned. Lastly, on the Caltech-101 Silhouettes dataset, VLAE demonstrated the lowest test NLL among the listed models.
=== LLM Response ===
The evaluation dataset used in Relational RNNs is the Semi-superv

In [221]:
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

invalid pdf header: b'<!DOC'


Getting tools for paper: papers/annotated_transformer.pdf


EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Failed to load file papers/annotated_transformer.pdf with error: RetryError[<Future at 0x2a2519f10 state=finished raised PdfStreamError>]. Skipping...
Getting tools for paper: papers/first_law_of_thermodynamics.html
Getting tools for paper: papers/karpathy_rnn_effectiveness.html
Getting tools for paper: papers/colah_understanding_lstms.html


invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Getting tools for paper: papers/cs231n_2015_rnn_regularization.pdf
Failed to load file papers/cs231n_2015_rnn_regularization.pdf with error: RetryError[<Future at 0x2a38e53d0 state=finished raised PdfStreamError>]. Skipping...
Getting tools for paper: papers/keeping_nn_simple_hinton.pdf
Getting tools for paper: papers/pointer_networks.pdf
Getting tools for paper: papers/imagenet_classification.pdf
Getting tools for paper: papers/order_matters_s_to_s.pdf
Getting tools for paper: papers/GPipe.pdf
Getting tools for paper: papers/deep_residual_learning_image_recognition.pdf
Getting tools for paper: papers/multi_scale_context_aggregation.pdf
Getting tools for paper: papers/nerual_quantum_chemistry.pdf
Getting tools for paper: papers/attention_is_all_you_need.pdf
Getting tools for paper: papers/neural_machine_translation.pdf
Getting tools for paper: papers/identity_mappings_deep_residual_networks.pdf
Getting tools for paper: papers/simple_nn_module.pdf
Getting tools for paper: papers/variati

In [222]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [223]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)


In [224]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [225]:
tools = obj_retriever.retrieve(
    "Tell me about the eval dataset used in MetaGPT and SWE-Bench"
)

In [226]:
tools[2].metadata

ToolMetadata(description='Use ONLY IF you want to get a holistic summary of MetaGPT. Do NOT use if you have specific questions over MetaGPT.', name='summary_tool_deep_speech_2', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [227]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm,
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)
agent = AgentRunner(agent_worker)

In [231]:
response = agent.query(
    "Explain RNNs for me"
)
print(str(response))

Added user message to memory: Explain RNNs for me
=== Calling Function ===
Calling function: summary_tool_cs231n_2015_rnn_regularization with args: {"input": "Recurrent Neural Networks (RNNs) are a type of neural network designed to handle sequential data. Unlike traditional feedforward neural networks, RNNs have connections that form a directed cycle, allowing them to exhibit dynamic temporal behavior. RNNs are well-suited for tasks such as language modeling, speech recognition, and time series prediction. However, they can suffer from issues like vanishing gradients, which can make it challenging for them to learn long-range dependencies. Various techniques like Long Short-Term Memory (LSTM) and Gated Recurrent Units (GRUs) have been developed to address these challenges and improve the effectiveness of RNNs in capturing long-term dependencies in sequential data."}
=== Function Output ===
Empty Response
=== LLM Response ===
RNNs (Recurrent Neural Networks) are a type of neural networ

In [237]:
response = agent.query(
    # "Compare and contrast the LoRA papers (LongLoRA, LoftQ). "
    # "Analyze the approach in each paper first. "
    "give me a list of papers that talk about lstms"
)

Added user message to memory: give me a list of papers that talk about lstms
=== LLM Response ===
Sure, I can help with that. Let me find a list of papers that talk about LSTMs for you.
=== Calling Function ===
Calling function: vector_tool_colah_understanding_lstms with args: {"query": "papers about LSTMs"}
=== Function Output ===
Kalchbrenner, Gregor, Chung, Bayer & Osendorfer are authors who have worked on papers related to LSTMs.
=== LLM Response ===
I found that authors like Kalchbrenner, Gregor, Chung, Bayer, and Osendorfer have worked on papers related to LSTMs. If you would like more specific information or details about these papers, please let me know.
